In [1]:
import numpy as np
import pandas as pd 

# geodata
from shapely.wkt import loads
import geopandas as gpd

# 거리계산
from geopy.distance import geodesic
from shapely.geometry import Point
from shapely.ops import unary_union
from sklearn.neighbors import BallTree
from sklearn.metrics.pairwise import haversine_distances
from math import radians

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 인코딩
from sklearn.preprocessing import LabelEncoder

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import yeojohnson

# 군집화
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import OPTICS
from sklearn.metrics import silhouette_score

# 증강
from imblearn.over_sampling import SMOTE

# 분류기
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

# 조합생성
from itertools import combinations

# 파라미터 탐색
from sklearn.model_selection import GridSearchCV

# 평가모델
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_squared_error, r2_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, confusion_matrix, accuracy_score

# 예측에 사용할 모델

Gradient Boosting  
{'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}


**최종 feature: 주택정보 + 대중교통 접근성**     

# 1. train 데이터 가공
- 화성시 데이터 가공

In [2]:
# 데이터 불러오기
gdf = gpd.read_file('../data/1-14.화성시_격자.geojson')
df_지하철= pd.read_csv('../data/1-6.화성시_지하철역.csv')
df_임대정보=pd.read_csv('../data/1-12.공공주택임대_정보(화성시).csv')
df_정류장=pd.read_csv('../data/1-5.화성시_버스정류장.csv')

# column명 변경
df_지하철.columns= ['역이름','지하철노선이름','경도','위도']
df_정류장.columns=['버스아이디','정류장이름','경도','위도']

# 거주인구
df_거주인구=pd.read_csv('../data/df_2022_임대주택거주인구.csv')
df_거주인구 = df_거주인구.drop(['월세', 
                              'm_20g_pop', 'w_20g_pop', 'm_30g_pop', 'w_30g_pop', 
                               'm_40g_pop', 'w_40g_pop', 'm_50g_pop', 'w_50g_pop', 
                               'm_60g_pop', 'w_60g_pop', 'm_70g_pop', 'w_70g_pop', 
                               'm_80g_pop', 'w_80g_pop', 'm_90g_pop', 'w_90g_pop',
                               'm_100g_pop', 'w_100g_pop'], axis = 1)
df_거주인구=df_거주인구.drop_duplicates()

## 1) 임대주택의 지원유형을 통합+거주인구 비율 추가 +평수 추가
- 세대수
- 두 가지 지원유형으로 바꾸기 => [공공임대, 통합공공임대]
- 거주인구 비율 추가
하나 알게 된점 : 거주인구의 79행과 공유차량 30행의 단지코드가 서로 알맞지 않다. 또한, 거주인구 데이터에 공유차량이 필요해보이는 행이 있었지만 배치되어 있지 않은듯 하다. 

In [3]:
# 단지평수 인코딩하여 열 추가하는 코드 
unique_gdf = df_거주인구.drop_duplicates(subset=['단지코드', '단지명', '지원유형', '세대수', '주차면수', '경도', '위도', 'geometry'])
dange_pyeongsu = df_거주인구.groupby('단지코드')['단지평수'].unique().reset_index()
dange_pyeongsu['단지평수'] = dange_pyeongsu['단지평수'].apply(lambda x: [str(i) for i in x])
dange_pyeongsu_encoded = dange_pyeongsu['단지평수'].str.join('|').str.get_dummies(sep='|')
dange_pyeongsu = pd.concat([dange_pyeongsu['단지코드'], dange_pyeongsu_encoded], axis=1)
final_gdf = unique_gdf.merge(dange_pyeongsu, on='단지코드', how='left')
df_거주인구=final_gdf.copy()

In [4]:
# 거주인구: 청년층 비율 계산 
df_거주인구['청년층비율']=df_거주인구.youth_pop/df_거주인구.total_pop

In [5]:
# 지원유형 통합 및 인코딩하여 추가
df_거주인구.loc[df_거주인구['지원유형'].isin(['영구임대', '국민임대', '행복주택']), '지원유형'] = '통합공공임대'
gdf_support_encoded = pd.get_dummies(df_거주인구['지원유형'], prefix='지원유형')
df_거주인구 = pd.concat([df_거주인구, gdf_support_encoded], axis=1)

In [6]:
# df_거주인구: 세대수, 평수, 지원유형, 지하철역거리, 청년층비율 추가
df_거주인구.columns

Index(['단지코드', '단지명', '지원유형', '세대수', '주차면수', '경도', '위도', 'geometry', '단지평수',
       'gid', 'total_pop', 'youth_pop', '10평 이하', '20평 이하', '30평 이하', '청년층비율',
       '지원유형_공공임대', '지원유형_통합공공임대'],
      dtype='object')

In [7]:
df_거주인구

,단지코드,단지명,지원유형,세대수,주차면수,경도,위도,geometry,단지평수,gid,total_pop,youth_pop,10평 이하,20평 이하,30평 이하,청년층비율,지원유형_공공임대,지원유형_통합공공임대
0,C00414,화성태안12,통합공공임대,1178,1767.0,127.041121,37.217786,POINT (127.04112089959335 37.217785871551655),20평 이하,"['다사593133', '다사592133', '다사594132', '다사591133...",1094.0,328.0,0,1,0,0.299817,0,1
1,C00415,화성태안6,통합공공임대,990,1485.0,127.049830,37.212454,POINT (127.04983042881659 37.212454460021775),20평 이하,"['다사599127', '다사600127', '다사601127', '다사600126...",1952.0,543.0,0,1,0,0.278176,0,1
2,C00416,화성태안8,통합공공임대,836,1254.0,127.047309,37.215054,POINT (127.04730900891269 37.215053662049854),20평 이하,"['다사597130', '다사599129', '다사598129', '다사597129...",2437.0,730.0,0,1,0,0.299549,0,1
3,C00447,화성매송(1단지)(국임),통합공공임대,649,973.5,126.880070,37.266522,POINT (126.88007016416084 37.266522352523815),20평 이하,"['다사451188', '다사449187', '다사450187', '다사450188...",1002.0,184.0,0,1,0,0.183633,0,1
4,C01566,화성동탄7-1(1-1)(7단지 능동),통합공공임대,682,1023.0,127.060256,37.216649,POINT (127.06025582642329 37.216649345765866),20평 이하,"['다사610130', '다사609131', '다사609132', '다사610131...",1477.0,554.0,0,1,0,0.375085,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,C02732,화성향남2 B-15블록 행복주택,통합공공임대,922,1383.0,126.906868,37.102767,POINT (126.90686788761191 37.10276739462884),10평 이하,"['다사472006', '다사472005', '다사473006', '다사473005...",856.0,257.0,1,1,0,0.300234,0,1
75,C02758,화성동탄2 A-53블록 행복주택,통합공공임대,700,1050.0,127.142678,37.187770,POINT (127.1426780536776 37.18777001201578),20평 이하,"['다사682101', '다사682097', '다사682100', '다사682098...",603.0,489.0,1,1,0,0.810945,0,1
76,C02784,화성비봉 A-5블록 행복·영구임대주택,통합공공임대,758,1137.0,126.866536,37.248863,POINT (126.86653578921877 37.248862760262405),20평 이하,"['다사438168', '다사437168', '다사438169', '다사437167']",657.0,163.0,1,1,0,0.248097,0,1
77,C02784,화성비봉 A-5블록 행복·영구임대주택,통합공공임대,136,204.0,126.866536,37.248863,POINT (126.86653578921877 37.248862760262405),10평 이하,"['다사438168', '다사437168', '다사438169', '다사437167']",116.0,30.0,1,1,0,0.258621,0,1


## 2) 지하철 + 정류장 추가

가장 가까운 정류장과의 거리  
가장 가까운 지하철과의 거리 

In [8]:
print(df_정류장.columns)
print(df_지하철.columns)

Index(['버스아이디', '정류장이름', '경도', '위도'], dtype='object')
Index(['역이름', '지하철노선이름', '경도', '위도'], dtype='object')


In [9]:
# 거리 계산 함수 (주어진 지점과 가장 가까운 거리 계산)
def calculate_min_distance(df_coords, target_coords):
    distances = np.array([
        geodesic(target_coords, station_coords).meters
        for station_coords in df_coords
    ])
    return distances.min()

# 정류장 또는 주차장의 위도/경도 좌표
df1_coords = df_정류장[['위도', '경도']].values  # 정류장 데이터
df3_coords = df_지하철[['위도', '경도']].values  # 지하철 데이터

# 아파트 단지의 위도/경도 좌표
apartments_coords = df_거주인구[['위도', '경도']].values

####### 1. 가장 가까운 정류장 거리 계산
def calculate_nearest_station(row):
    target_coords = (row['위도'], row['경도'])
    distances = haversine_distances(
        [np.radians(target_coords)], np.radians(df1_coords)
    )[0] * 6371000  # 미터로 변환
    return distances.min()
def calculate_nearest_subway(row):
    target_coords = (row['위도'], row['경도'])
    distances = haversine_distances(
        [np.radians(target_coords)], np.radians(df3_coords)
    )[0] * 6371000  # 미터로 변환
    return distances.min()

df_거주인구['nearest_station_distance'] = df_거주인구.apply(calculate_nearest_station, axis=1)
df_거주인구['nearest_subway_station_distance'] = df_거주인구.apply(calculate_nearest_subway, axis=1)

In [10]:
df_거주인구.columns

Index(['단지코드', '단지명', '지원유형', '세대수', '주차면수', '경도', '위도', 'geometry', '단지평수',
       'gid', 'total_pop', 'youth_pop', '10평 이하', '20평 이하', '30평 이하', '청년층비율',
       '지원유형_공공임대', '지원유형_통합공공임대', 'nearest_station_distance',
       'nearest_subway_station_distance'],
      dtype='object')

## 최종 train set

In [11]:
df=df_거주인구.drop(['단지코드', '단지명', '지원유형', '주차면수', '경도','위도', 'geometry', '단지평수','gid', '지원유형_공공임대'],axis=1)
df.columns

Index(['세대수', 'total_pop', 'youth_pop', '10평 이하', '20평 이하', '30평 이하', '청년층비율',
       '지원유형_통합공공임대', 'nearest_station_distance',
       'nearest_subway_station_distance'],
      dtype='object')

# 2. test 데이터 가공
- 하남시 데이터 가공

In [12]:
# 데이터 불러오기
df_지하철_하남= pd.read_csv('../data/타겟/하남_지하철.csv')
df_임대정보_하남=pd.read_csv('../data/타겟/교산_신설공동주택정보.csv')
df_정류장_하남=pd.read_csv('../data/타겟/2-5.하남시_버스정류장.csv')
df_단지정보_하남 = pd.read_csv('../data/타겟/교산_주택공급.csv')
df_격자 = pd.read_csv('../data/타겟/하남시_격자별_블록코드.csv')

# 데이터 정리
df_임대정보_하남 = df_임대정보_하남.drop(['Unnamed: 0','area','세대수'], axis=1)
df_격자 = df_격자.drop(columns='blck_cd')

# column명 변경
df_임대정보_하남.rename({'blockName':'blck_cd'},axis=1,inplace=True)
df_단지정보_하남.rename({'통합공임여부':'지원유형_통합공공임대'},axis=1,inplace=True)
df_지하철_하남.columns= ['역이름','경도','위도']
df_정류장_하남.columns=['버스아이디','정류장이름','경도','위도']

In [13]:
# 문자열을 지리 데이터로 변환 후, GeoDataFrame으로 변환
df_임대정보_하남['geometry'] = df_임대정보_하남['geometry'].apply(loads)
df_격자['geometry'] = df_격자['geometry'].apply(loads)
df_임대정보_하남 = gpd.GeoDataFrame(df_임대정보_하남, geometry='geometry')
df_격자 = gpd.GeoDataFrame(df_격자, geometry='geometry')

## 1) 임대주택 지원유형을 통합 + 평수 데이터 생성

In [14]:
# 단지정보 + 임대정보 병합
df_공공주택_하남 = pd.merge(df_단지정보_하남,df_임대정보_하남,on = ['blck_cd','10평이상','20평이상','30평이상'], how = 'outer')

# 변수명 정의 및 drop
df_공공주택_하남.rename({'건설호수':'세대수'},axis=1,inplace=True)
df_공공주택_하남.drop('통합공임 포함',axis=1, inplace=True)

# LH임대주택인 것만 포함
df_공공주택_하남 = df_공공주택_하남.iloc[:-2,:] 

# 중복 제거
df_공공주택_하남 = df_공공주택_하남.drop_duplicates().reset_index(drop=True)

In [15]:
# 단지 평수에 대한 인코딩 변환
# 이상 → 이하로 변환
df_공공주택_하남['10평 이하'] = 0  # 10평 이하는 모두 0으로 설정
df_공공주택_하남['20평 이하'] = df_공공주택_하남['10평이상']  # 10평 이상을 20평 이하로
df_공공주택_하남['30평 이하'] = df_공공주택_하남['20평이상']  # 20평 이상을 30평 이하로

# 기존 열 삭제
df_공공주택_하남 = df_공공주택_하남.drop(['10평이상', '20평이상', '30평이상'], axis=1)

In [16]:
# 중심점 계산
def calculate_centroid_ha(geometry):
    return geometry.centroid
# 미터 거리 계산
def calculate_distance_ha(coord1, coord2):
    return geodesic(coord1, coord2).meters

# 위도 경도 변수 생성
df_공공주택_하남['centroid'] = df_공공주택_하남['geometry'].apply(calculate_centroid_ha)
df_공공주택_하남['경도'] = df_공공주택_하남['centroid'].apply(lambda x: x.x)
df_공공주택_하남['위도'] = df_공공주택_하남['centroid'].apply(lambda x: x.y)

df_공공주택_하남

,blck_cd,세대수,지원유형_통합공공임대,geometry,10평 이하,20평 이하,30평 이하,centroid,경도,위도
0,S1,333,0,"POLYGON ((127.22083 37.53749, 127.22135 37.536...",0,0,1,POINT (127.22085463797028 37.53654240083496),127.220855,37.536542
1,A1,965,0,"POLYGON ((127.21965 37.53860, 127.21967 37.538...",0,1,0,POINT (127.21945047728771 37.53791708557968),127.219450,37.537917
2,A2,1115,0,"POLYGON ((127.21992 37.53696, 127.21880 37.535...",0,0,1,POINT (127.21814866827263 37.53690350143911),127.218149,37.536904
3,A3,263,0,"POLYGON ((127.22110 37.53566, 127.22022 37.534...",0,0,1,POINT (127.22002314039142 37.53572555502718),127.220023,37.535726
4,A3,131,1,"POLYGON ((127.22110 37.53566, 127.22022 37.534...",0,0,1,POINT (127.22002314039142 37.53572555502718),127.220023,37.535726
5,A4,666,1,"POLYGON ((127.22546 37.53456, 127.22504 37.533...",0,1,0,POINT (127.22438304433037 37.53413761941045),127.224383,37.534138
6,A5,492,0,"POLYGON ((127.22504 37.53327, 127.22456 37.531...",0,0,1,POINT (127.22399383359954 37.53288787002548),127.223994,37.532888
7,A6,513,0,"POLYGON ((127.22445 37.53154, 127.22442 37.531...",0,0,1,POINT (127.22303873200528 37.531549398026215),127.223039,37.531549
8,A6,256,1,"POLYGON ((127.22445 37.53154, 127.22442 37.531...",0,0,1,POINT (127.22303873200528 37.531549398026215),127.223039,37.531549
9,A7,1605,1,"POLYGON ((127.20894 37.52430, 127.20896 37.524...",0,1,0,POINT (127.20791108747989 37.52356108585517),127.207911,37.523561


## 2) 지하철 + 정류장 추가

가장 가까운 정류장과의 거리  
가장 가까운 지하철과의 거리 

In [17]:
print(df_정류장_하남.columns)
print(df_지하철_하남.columns)

Index(['버스아이디', '정류장이름', '경도', '위도'], dtype='object')
Index(['역이름', '경도', '위도'], dtype='object')


In [18]:
# 거리 계산 함수 (주어진 지점과 가장 가까운 거리 계산)
def calculate_min_distance(df_coords, target_coords):
    distances = np.array([
        geodesic(target_coords, station_coords).meters
        for station_coords in df_coords
    ])
    return distances.min()

# 정류장 또는 주차장의 위도/경도 좌표
df1_coords = df_정류장_하남[['위도', '경도']].values  # 정류장 데이터
df3_coords = df_지하철_하남[['위도', '경도']].values  # 지하철 데이터

# 아파트 단지의 위도/경도 좌표
apartments_coords = df_공공주택_하남[['위도', '경도']].values

####### 1. 가장 가까운 정류장 거리 계산
def calculate_nearest_station(row):
    target_coords = (row['위도'], row['경도'])
    distances = haversine_distances(
        [np.radians(target_coords)], np.radians(df1_coords)
    )[0] * 6371000  # 미터로 변환
    return distances.min()
def calculate_nearest_subway(row):
    target_coords = (row['위도'], row['경도'])
    distances = haversine_distances(
        [np.radians(target_coords)], np.radians(df3_coords)
    )[0] * 6371000  # 미터로 변환
    return distances.min()

df_공공주택_하남['nearest_station_distance'] = df_공공주택_하남.apply(calculate_nearest_station, axis=1)
df_공공주택_하남['nearest_subway_station_distance'] = df_공공주택_하남.apply(calculate_nearest_subway, axis=1)

## 최종 test set

In [19]:
df_test=df_공공주택_하남.drop(['blck_cd', 'geometry', 'centroid', '경도','위도'],axis=1)
df_test.columns

Index(['세대수', '지원유형_통합공공임대', '10평 이하', '20평 이하', '30평 이하',
       'nearest_station_distance', 'nearest_subway_station_distance'],
      dtype='object')

## feature, target 변수 설정

*** feature ***  
주택정보 : '세대수',  '10평 이하', '20평 이하', '30평 이하',   '지원유형_통합공공임대'  
대중교통 접근성 : 'nearest_station_distance','nearest_subway_station_distance'    

*** target ***  
청년층강도: 청년층비율 > 0.3 ⇨ 1

In [20]:
# target 생성
def categorize_ratio(청년층비율):
    if 청년층비율 <= 0.3:
        return 0
    else:
        return 1

# 데이터 복사
df2 = df.copy()

df2['청년층강도'] = df2['청년층비율'].apply(categorize_ratio)
print(df2['청년층강도'].value_counts())

1    43
0    36
Name: 청년층강도, dtype: int64


In [21]:
# train set: 화성시 데이터
X_train = df2.drop(columns=['total_pop','youth_pop', '청년층강도', '청년층비율'])
y_train = df2['청년층강도']  # 타겟 변수

In [22]:
# test set: 하남시(교산지구) 데이터
X_test = df_test

# 증강

In [23]:
# train 데이터에 대해 데이터 증강 수행
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [24]:
print(y_train.value_counts())
print(y_train_resampled.value_counts())

1    43
0    36
Name: 청년층강도, dtype: int64
1    43
0    43
Name: 청년층강도, dtype: int64


In [25]:
# 모델 생성 및 학습
gb_model = GradientBoostingClassifier(learning_rate=0.2, max_depth=7, n_estimators=200, subsample=0.8, random_state=42)
gb_model.fit(X_train_resampled, y_train_resampled)

# 교산지구 예측
y_pred_하남 = gb_model.predict(X_test)
y_pred_하남_series = pd.Series(y_pred_하남, name="청년층강도_pred")

In [26]:
df_교산지구 = pd.concat([df_공공주택_하남,y_pred_하남_series],axis=1)
df_교산지구.to_csv('../data/타겟/교산지구_거주인구_예측.csv',index=False)

In [27]:
df_교산지구

,blck_cd,세대수,지원유형_통합공공임대,geometry,10평 이하,20평 이하,30평 이하,centroid,경도,위도,nearest_station_distance,nearest_subway_station_distance,청년층강도_pred
0,S1,333,0,"POLYGON ((127.22083 37.53749, 127.22135 37.536...",0,0,1,POINT (127.22085463797028 37.53654240083496),127.220855,37.536542,266.107317,424.362560,0
1,A1,965,0,"POLYGON ((127.21965 37.53860, 127.21967 37.538...",0,1,0,POINT (127.21945047728771 37.53791708557968),127.219450,37.537917,104.186866,407.392405,0
2,A2,1115,0,"POLYGON ((127.21992 37.53696, 127.21880 37.535...",0,0,1,POINT (127.21814866827263 37.53690350143911),127.218149,37.536904,149.532509,564.678226,0
3,A3,263,0,"POLYGON ((127.22110 37.53566, 127.22022 37.534...",0,0,1,POINT (127.22002314039142 37.53572555502718),127.220023,37.535726,307.697383,540.526743,0
4,A3,131,1,"POLYGON ((127.22110 37.53566, 127.22022 37.534...",0,0,1,POINT (127.22002314039142 37.53572555502718),127.220023,37.535726,307.697383,540.526743,0
5,A4,666,1,"POLYGON ((127.22546 37.53456, 127.22504 37.533...",0,1,0,POINT (127.22438304433037 37.53413761941045),127.224383,37.534138,115.789900,630.538673,1
6,A5,492,0,"POLYGON ((127.22504 37.53327, 127.22456 37.531...",0,0,1,POINT (127.22399383359954 37.53288787002548),127.223994,37.532888,216.614294,765.584148,0
7,A6,513,0,"POLYGON ((127.22445 37.53154, 127.22442 37.531...",0,0,1,POINT (127.22303873200528 37.531549398026215),127.223039,37.531549,294.989939,913.582519,0
8,A6,256,1,"POLYGON ((127.22445 37.53154, 127.22442 37.531...",0,0,1,POINT (127.22303873200528 37.531549398026215),127.223039,37.531549,294.989939,913.582519,0
9,A7,1605,1,"POLYGON ((127.20894 37.52430, 127.20896 37.524...",0,1,0,POINT (127.20791108747989 37.52356108585517),127.207911,37.523561,96.145746,643.974702,1


In [28]:
y_pred_하남

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0])